<a href="https://colab.research.google.com/github/JSJeong-me/CLIP-Zero-Shot-Classification/blob/main/10class_CLIP_Zero_Shot_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use CLIP Zero-Shot on your own classificaiton dataset

This notebook provides an example of how to benchmark CLIP's zero shot classification performance on your own classification dataset.

[CLIP](https://openai.com/blog/clip/) is a new zero shot image classifier relased by OpenAI that has been trained on 400 million text/image pairs across the web. CLIP uses these learnings to make predicts based on a flexible span of possible classification categories.

CLIP is zero shot, that means **no training is required**. 

Try it out on your own task here!

Be sure to experiment with various text prompts to unlock the richness of CLIP's pretraining procedure.


![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)


# Download and Install CLIP Dependencies

In [1]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

import numpy as np
import torch

print("Torch version:", torch.__version__)

CUDA version: 11.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:23tcmalloc: large alloc 1147494400 bytes == 0x561ef24f6000 @  0x7f6383ca5615 0x561eef86c4cc 0x561eef94c47a 0x561eef86f2ed 0x561eef960e1d 0x561eef8e2e99 0x561eef8dd9ee 0x561eef870bda 0x561eef8e2d00 0x561eef8dd9ee 0x561eef870bda 0x561eef8df737 0x561eef961c66 0x561eef8dedaf 0x561eef961c66 0x561eef8dedaf 0x561eef961c66 0x561eef8dedaf 0x561eef871039 0x561eef8b4409 0x561eef86fc52 0x561eef8e2c25 0x561eef8dd9ee 0x561eef870bda 0x561eef8df737 0x561eef8dd9ee 0x561eef870bda 0x561eef8de915 0x561eef870afa 0x561eef8dec0d 0x561eef8dd9ee
     |█████████████████████████████▏  | 1055.7 MB 65.9 MB/s eta 0:00:02tcmalloc: large alloc 1434370048 bytes == 0x561f36b4c000 @  0x7f6383ca5615 0x561eef86c4cc 0x561eef94c47a 0x561eef86f2ed 0x561eef960e1d 0x561eef8e2e99 0x561eef8dd9ee 0x561eef870bda 0x561eef8e2d00 0x561eef8dd9ee 0x561eef870bda 0x561eef8df737 0x561ee

In [2]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 164 (delta 33), reused 55 (delta 29), pack-reused 91
Receiving objects: 100% (164/164), 8.91 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/content/CLIP


# Download Classification Data or Object Detection Data

We will download the [public flowers classificaiton dataset](https://public.roboflow.com/classification/flowers_classification) from Roboflow. The data will come out as folders broken into train/valid/test splits and seperate folders for each class label.

You can easily download your own dataset from Roboflow in this format, too.

We made a conversion from object detection to CLIP text prompts in Roboflow, too, if you want to try that out.


To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd ./CLIP

/content/CLIP


In [ ]:
!cp /content/drive/MyDrive/Flowers_Classification.v3-augmented.clip.zip .

In [11]:
!pwd

/content/CLIP


In [ ]:
!unzip ./vit01.v2i.clip.zip

In [5]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./test/')
class_names.remove('_tokenization.txt')
class_names

['four', 'car', 'fish', 'bench', 'apple', 'dog']

In [16]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./test/_tokenization.txt

An example picture from the fruits dataset depicting a apple
An example picture from the car dataset depicting a car

In [9]:
#edit your prompts as you see fit here
%%writefile ./test/_tokenization.txt
An example picture from the flowers dataset depicting a daisy
An example picture from the flowers dataset depicting a dandelion

Overwriting ./test/_tokenization.txt


In [17]:
candidate_captions = []
with open('./test/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [18]:
candidate_captions

['An example picture from the fruits dataset depicting a apple',
 'An example picture from the car dataset depicting a car']

In [17]:
%cd ./CLIP

/content/CLIP


# Run CLIP inference on your classification dataset

In [25]:
!ls -l

total 12
drwxr-xr-x 8 root root 4096 Oct 18 13:24 CLIP
-rw-r--r-- 1 root root  402 Oct 18 13:27 README.roboflow.txt
drwxr-xr-x 1 root root 4096 Oct  8 13:45 sample_data


In [22]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./test/*.jpg')
    print(test_imgs)
    for img in test_imgs:
        print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    #print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
    print('accuracy on class ' + cls + ' is :{}   {}'.format(str(sum(class_correct)), str(len(class_correct))))
print('accuracy on class ' + cls + ' is :{}   {}'.format(str(sum(class_correct)), str(len(class_correct))))

[]
accuracy on class four is :0   0
[]
accuracy on class car is :0   0
[]
accuracy on class fish is :0   0
[]
accuracy on class bench is :0   0
[]
accuracy on class apple is :0   0
[]
accuracy on class dog is :0   0
accuracy on class dog is :0   0
